In [1]:
import pandas as pd
import polars as pl
from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.utils import shuffle

# КЛАССИФИКАЦИЯ ТЕКСТОВ

# Задача:
## Тестовый файл содержит 1000 объектов и представлен по ссылке 
## (https://courses.openedu.ru/assets/courseware/v1/a4f631b0af5d3b3cc07eb638709f37ef/asset-v1:ITMOUniversity+SOCNETAN+spring_2024_ITMO_bac+type@asset+block/Soc_Net_Task_2_test_16.zip)

### Введите полученные предсказания для всех объектов
#### Предсказания вводить через запятую, например 0,1,0,1,1 (тысяча значений)

# Подготовка тестовых данных

In [2]:
test_df = pd.read_csv('Soc_Net_Task_2_test_16.csv')
test_df

,id,text
0,207914,есть инфа что вы завтра будете жрать хуевую еду
1,205429,RT MarKoroleva В объявлении о наборе стюардесс...
2,224471,RT F1Popov И госпиталь в Гренобле и менеджер Ш...
3,210990,TorreyMcAllen Меня пугает такое отношение к Дж...
4,216561,Джину для того чтобы стать свободным понадобил...
...,...,...
995,209032,DiankaPavlenko это секретные данные КМНУтолько...
996,207567,Что за лаккидень 3 зачет и автомат получила а ...
997,215403,AliLoginova я тут взяла тетрадичтобы переписат...
998,209972,advakhova прости \nвеселье уедет в раменское...


In [3]:
test_df.id.nunique()

1000

In [4]:
x_test = test_df['text']

# Подготовка тренировочных данных

In [5]:
train_df = pl.read_csv('train.csv').to_pandas()
train_df = train_df[['text', 'class']]
train_df

,text,class
0,VitaliaPerilova у тебя классные очкиони тебя о...,1
1,Никогда не подумала бы что самое приятное на с...,1
2,KlimovaElena о дааа вот это я понимаю как никт...,0
3,Говорят еле хожу никого не вижу будто живой ме...,0
4,то чувство когда понимаешь что завтра можно вы...,1
...,...,...
181462,CATAHAHELENA я просто смотрела много серий под...,0
181463,И кстати девчонки когда вышла вечером пробежат...,1
181464,AnnaPolitova trulyalya за борсучка обидно,0
181465,Жалко Березовского Горький конец Но счастье п...,0


In [6]:
train_df.dtypes

text     object
class     int64
dtype: object

In [7]:
train_df.isna().sum()

text     0
class    0
dtype: int64

In [8]:
x_train = train_df['text']
y_train = train_df['class']

In [9]:
x_train, y_train = shuffle(x_train, y_train, random_state=0)

# Подготовка валидационных данных

In [10]:
val_df = pl.read_csv('val.csv').to_pandas()
val_df = val_df[['text', 'class']]
val_df

,text,class
0,tanchss простудился лежу уже второй день,0
1,KEMbIHAKEMAX у меня не лучше посмотри что ль н...,0
2,Лежу под капельницей httptcoEsXUd5Efrg,0
3,ChristyZobnina плачет маленькая Алла ведь забр...,0
4,RT egyxesutra httptcokhJ1RYa1Ku Бывают такие п...,0
...,...,...
22678,RT bridledjoy imustdie зажрался httptcoHN2gJerude,0
22679,Как бесит когда находишь бит с ахуенным сэмпл...,0
22680,RT jypuhofilepe ходил сегодня на футбол На мат...,1
22681,RT sochibig SofaZelik красота Хорошего воскрес...,1


In [11]:
val_df.dtypes

text     object
class     int64
dtype: object

In [12]:
val_df.isna().sum()

text     0
class    0
dtype: int64

In [13]:
x_val = val_df['text']
y_val = val_df['class']

In [14]:
x_val, y_val = shuffle(x_val, y_val, random_state=0)

# Настройка модели

In [15]:
tfidf = TfidfVectorizer()

In [16]:
# обучаем векторизатор на тренировочной выборке и преобразуем в числовой формат
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(x_train)
X_train_tfidf = tfidf.transform(x_train)

In [17]:
# обучаем векторизатор на валидационной выборке и преобразуем в числовой формат
# tfidf = TfidfVectorizer()
# X_val_tfidf = tfidf.fit_transform(x_val)
X_val_tfidf = tfidf.transform(x_val)

In [18]:
# обучаем векторизатор на тестовой выборке и преобразуем в числовой формат
# tfidf = TfidfVectorizer()
# X_test_tfidf = tfidf.fit_transform(x_test)
X_test_tfidf = tfidf.transform(x_test)

In [19]:
%%time
model_text = MultinomialNB()
model_text.fit(X_train_tfidf, y_train)

CPU times: user 65.4 ms, sys: 79 µs, total: 65.5 ms
Wall time: 70.5 ms


MultinomialNB()

In [20]:
y_pred = model_text.predict(X_val_tfidf)

In [21]:
accuracy = accuracy_score(y_val, y_pred)
accuracy

0.7471674822554336

In [22]:
print(y_pred)

[1 0 0 ... 1 1 1]


In [23]:
stroka = """1 1 0 1 1 1 1 1 1 0 1 1 0 1 1 1 0 0 1 0 0 0 0 1 0 1 0 0 0 0 1 0 1 1 0 0 1
 0 0 1 1 0 0 1 0 1 0 0 0 1 1 1 0 0 0 1 0 1 0 1 0 0 0 0 1 1 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 1 0 1 1 0 1 1 1 1 0 1 1 1 0 0 1 1 0 0 1 0
 1 0 0 0 0 1 0 1 0 0 1 1 0 0 0 1 1 1 0 0 1 1 1 1 0 0 1 1 1 1 1 0 0 0 0 0 1
 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 1 0 0 1 1 1 1 1
 0 0 1 0 1 1 1 0 1 0 1 1 0 1 0 0 1 0 1 0 1 0 1 1 1 0 1 0 0 1 0 1 1 0 0 1 1
 1 1 0 1 0 0 1 1 1 0 0 1 1 1 0 0 1 0 1 0 0 1 1 0 1 1 0 0 1 1 0 0 1 1 0 1 1
 1 1 1 0 0 1 0 1 0 0 1 1 1 0 0 0 1 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 1 0 1 1 1
 1 1 0 0 0 0 0 1 0 1 0 1 0 1 0 1 1 1 1 1 1 1 0 1 0 0 1 0 1 0 1 1 1 1 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 1 1 1 0 0 1 1 1 0 1 1 0 1 1 0 1 1 0 1 1 1 1 0 0 1
 0 1 1 1 1 0 0 0 1 1 0 0 0 0 1 0 0 0 0 1 1 0 1 0 1 0 1 1 1 1 0 0 0 0 0 0 1
 0 0 1 0 1 0 1 1 0 0 1 1 0 1 0 0 0 1 0 0 1 1 1 1 0 0 1 1 1 0 1 0 0 0 1 1 1
 0 0 1 0 1 1 0 1 0 0 1 1 1 1 1 0 1 0 1 1 1 0 1 1 0 0 1 0 0 0 1 1 0 1 1 0 1
 1 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 0 0 0 1 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 0 0 0 0 1 1 0 0
 0 1 1 1 1 0 0 1 1 1 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 1 0 1 1 1 1 0 1 0 0 1 1
 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 1 1 1 1 0 1 0 1 1 0 0 1 0 0 1 0
 0 1 0 0 0 0 1 0 1 1 0 0 0 1 1 1 0 0 1 0 0 1 1 1 1 0 1 1 0 0 0 0 0 1 0 1 1
 1 1 1 1 0 0 0 0 0 1 0 1 1 1 1 0 1 1 1 0 1 0 1 0 1 1 0 0 1 1 0 1 0 0 1 1 1
 1 0 1 0 0 0 1 0 0 0 0 0 1 0 1 1 1 0 1 1 1 1 0 0 0 0 1 0 0 0 0 0 1 0 1 0 1
 0 0 0 0 0 0 1 0 0 0 1 0 0 1 1 0 1 0 0 1 1 0 0 1 0 1 0 1 1 0 1 0 1 1 0 0 0
 1 0 1 0 0 0 1 0 1 0 0 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 0 1 1 1 0 1 1 0 1 0 1
 0 0 0 1 0 0 1 0 0 1 0 1 1 0 1 1 0 0 0 1 0 0 1 0 1 0 0 0 0 1 0 0 0 0 1 0 0
 1 1 1 1 0 0 0 1 0 1 1 1 0 0 0 1 1 0 0 1 0 0 1 1 0 0 0 0 1 0 1 1 0 0 1 0 1
 1 0 1 0 1 1 1 0 1 1 0 0 1 1 0 1 0 0 0 0 0 0 1 0 1 0 0 1 0 0 1 0 1 0 0 1 0
 1 0 0 1 0 0 1 0 0 0 1 1 0 1 1 1 0 0 1 0 1 0 0 1 0 1 1 0 1 1 0 0 0 0 0 1 1
 0 0 0 0 1 1 0 0 0 1 0 1 1 0 0 0 1 1 0 1 0 0 0 0 1 1 1 0 0 0 1 0 0 1 1 1 0
 0"""""

In [24]:
# копируем - вставляем в ответ - получаем 1 балл
stroka.replace(" ", ",").replace("\n", "")

'1,1,0,1,1,1,1,1,1,0,1,1,0,1,1,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,1,1,0,0,1,0,0,1,1,0,0,1,0,1,0,0,0,1,1,1,0,0,0,1,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,1,0,1,1,0,1,1,1,1,0,1,1,1,0,0,1,1,0,0,1,0,1,0,0,0,0,1,0,1,0,0,1,1,0,0,0,1,1,1,0,0,1,1,1,1,0,0,1,1,1,1,1,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,1,0,1,0,0,0,1,0,0,1,1,1,1,1,0,0,1,0,1,1,1,0,1,0,1,1,0,1,0,0,1,0,1,0,1,0,1,1,1,0,1,0,0,1,0,1,1,0,0,1,1,1,1,0,1,0,0,1,1,1,0,0,1,1,1,0,0,1,0,1,0,0,1,1,0,1,1,0,0,1,1,0,0,1,1,0,1,1,1,1,1,0,0,1,0,1,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1,0,0,1,1,1,0,0,1,0,1,0,1,1,1,1,1,0,0,0,0,0,1,0,1,0,1,0,1,0,1,1,1,1,1,1,1,0,1,0,0,1,0,1,0,1,1,1,1,0,0,0,1,1,1,1,0,0,1,0,1,1,0,1,1,1,1,0,0,1,1,1,0,1,1,0,1,1,0,1,1,0,1,1,1,1,0,0,1,0,1,1,1,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,1,0,1,0,1,0,1,1,1,1,0,0,0,0,0,0,1,0,0,1,0,1,0,1,1,0,0,1,1,0,1,0,0,0,1,0,0,1,1,1,1,0,0,1,1,1,0,1,0,0,0,1,1,1,0,0,1,0,1,1,0,1,0,0,1,1,1,1,1,0,1,0,1,1,1,0,1,1,0,0,1,0,0,0,1,1,0,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0